In [ ]:
import json


def read_json_file(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data


def parse_json(json_string):
    json_dict = json.loads(json_string)
    return json_dict


def prettify_json(obj):
    pretty_json = json.dumps(obj, indent=2)
    return pretty_json


def write_to_json_file(obj, file_path):
    with open(file_path, 'w') as json_file:
        json.dump(obj, json_file, indent=2)


In [ ]:
import configparser

def read_ini_file(file_path):
    config = configparser.ConfigParser()
    config.read(file_path)
    ini_dict = {section: dict(config.items(section))
                for section in config.sections()}
    return ini_dict


In [ ]:
def transform_graph(graph):
	nodes = { node['data']['id']: node['data'] for node in graph['elements']['nodes'] }
	edges = {}
	for edge in graph['elements']['edges']:
		if 'label' in edge['data']:
			label = edge['data']['label']
		else:
			label = ','.join(edge['data']['labels'])
			edge['data']['label'] = label
		
		if label not in edges:
			edges[label] = []
		edges[label].append(edge['data'])
	return (nodes, edges)
	

In [ ]:
config = read_ini_file('config.ini')
project_name = config['project']['name']
project_desc = config['project']['desc']
ifile = config['project']['ifile']
(project_name,project_desc,ifile)

In [ ]:
graph = read_json_file(ifile)
nodes,edges = transform_graph(graph)

list(nodes.keys()), list(edges.keys())

In [ ]:
node_header = ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")
edge_header = ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")

In [ ]:
domain_name = f'{project_name}'
domain_node = (domain_name, "Domain", domain_name, domain_name, "#666666", None, None)



In [ ]:
nodes,edges = transform_graph(graph)

In [ ]:
list(nodes.keys())

In [ ]:
list(edges.keys())

In [ ]:
def extract_top_level_packages(data):
    # Remove the last element from each tuple and convert to set for uniqueness
    unique_prefixes = set(tuple(item[:-1]) for item in data)
    
    # Sort unique_prefixes by length of tuples in ascending order
    sorted_prefixes = sorted(unique_prefixes, key=len)

    results = []

    for prefix in sorted_prefixes:
        # Check if the prefix is already a prefix of any result
        if not any(prefix[:len(result)] == result for result in results):
            results.append(prefix)

    return results

In [ ]:
def find_path_from_root(tree, target_node):
    # Step 1: Build a dictionary to map each node to its parent
    parent_map = {}
    for edge in tree:
        parent_map[edge['target']] = edge['source']
    
    # Step 2: Trace the path from target_node to the root
    path = []
    current_node = target_node
    while current_node in parent_map:
        path.append(current_node)
        current_node = parent_map[current_node]
    
    # Step 3: Append the root node to the path
    if current_node is not None:
        path.append(current_node)
    
    # Step 4: Reverse the path to get root to target_node order
    path.reverse()
    
    return tuple(path)


In [ ]:
pkg_with_classes = {edge['source'] for edge in edges['contains'] if 'Container' in nodes[edge['source']]['labels'] and 'Structure' in nodes[edge['target']]['labels']}
pkg_with_classes

In [ ]:
pkg_paths = [find_path_from_root(edges['contains'], pkg_id) for pkg_id in pkg_with_classes]
sorted(pkg_paths)

In [ ]:
top_level_packages = extract_top_level_packages(pkg_paths)
top_level_packages

In [ ]:
def create_mapping(list1, list2):
    mapping = dict()
    for tup in list1:
        key = tup[-1]  # Last element of the tuple as the key
        for tup2 in list2:
            if tuple(tup[:len(tup2)]) == tup2:  # Match the prefix part in list1 with list2
                mapping[key] = tup2[-1]
                break
    return mapping

In [ ]:
sublayer_to_component = create_mapping(pkg_paths, top_level_packages)
sublayer_to_component

In [ ]:
components = [pkg[-1] for pkg in top_level_packages]
components

In [ ]:
contains = []
component_nodes = []
for component in components:
    component_props = nodes[component]['properties']
    component_node = (component, "Component", component, component, "#666666", None, None)
    component_nodes.append(component_node)
    contains.append((f"{domain_name}-contains-{component}", "CONTAINS", domain_name, component, "{}", None, None, None))
contains

In [ ]:
layerColors = {	
    'Presentation Layer': '#ee3239',
	'Service Layer': '#fece00',
	'Domain Layer': '#5eaa5f',
	'Data Source Layer': '#6a6dba',
	'Unknown': '#666666'
}

In [ ]:
# ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")

# packages that directly contain classes
sublayers = list(set((pkg, 'Sublayer', pkg, pkg, layerColors[nodes[pkg]['properties'].get('layer', 'Unknown')], None, None)
                     for pkg in pkg_with_classes))

sublayers

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
contains += [(f'{sublayer_to_component[id]}-contains-{id}','CONTAINS',sublayer_to_component[id],id,"{}",None,None,None) for id,label,fullName,simpleName,color,depProfileCat,cohesion in sublayers]

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
contains += [(edge['id'],'CONTAINS',edge['source'],edge['target'],"{}",None,None,None)
	for edge in edges['contains'] if 'Container' in nodes[edge['source']]['labels'] and 'Structure' in nodes[edge['target']]['labels']]

contains

In [ ]:
# ("id",":TYPE",":START_ID",":END_ID","references","dependencyTypes","nrDependencies:INT","nrCalls:INT")
calls = [(f'{edge["source"]}-calls-{edge["target"]}','CALLS',edge['source'],edge['target'],"{}","compile_time",edge['properties']['weight'],None)
	for edge in edges['calls'] if edge['source'] != edge['target']]

calls

In [ ]:
def invert(edgeList):
    prefix = "inv_"
    invertedEdges = []
    for edge in edgeList:
        invertedEdge = {
            'source': edge['target'],
            'target': edge['source'],
            'label': prefix + edge.get('label', ''),
            **{key: value for key, value in edge.items() if key not in ['source', 'target', 'label']}
        }
        invertedEdges.append(invertedEdge)
    return invertedEdges

In [ ]:

parents = {e['source']:e['target'] for e in invert(edges['contains'])}
parents

In [ ]:
from collections import Counter

dependencyProfiles = dict()

for edge in edges['calls']:

	srcId = edge['source']
	tgtId = edge['target']

	if not srcId in dependencyProfiles:
		dependencyProfiles[srcId] = list()
	if not tgtId in dependencyProfiles:
		dependencyProfiles[tgtId] = list()
	
	if parents[srcId] != parents[tgtId]:
		dependencyProfiles[srcId].append('out')
		dependencyProfiles[tgtId].append('in')

dependencyProfiles = {id:Counter(profile) for id, profile in dependencyProfiles.items()}

In [ ]:
def depProfile(inn,out):
	if inn==0 and out>0:
		return "outbound"
	elif inn>0 and out==0:
		return "inbound"
	elif inn>0 and out>0:
		return "transit"
	else:
		return "hidden"

In [ ]:
dependencyProfiles = {id:depProfile(profile['in'],profile['out']) 
		for id, profile in dependencyProfiles.items()}

In [ ]:
dependencyProfiles

In [ ]:

projProfile = Counter(dependencyProfiles.values())
projProfile

In [ ]:
l = (projProfile['hidden'],projProfile['inbound'],projProfile['outbound'],projProfile['transit'])
n = sum(l)
tuple(i/n for i in l)

In [ ]:
roleStereotypeColors = {
    "Unknown": "#cccccc",
    "Controller": "#decbe4",
    "Coordinator": "#ccebc5",
    "Information Holder": "#fbb4ae",
    "Interfacer": "#fed9a6",
    "User Interfacer": "#fed9a6",
    "Internal Interfacer": "#fed9a6",
    "External Interfacer": "#fed9a6",
    "Service Provider": "#b3cde3",
    "Structurer": "#fddaec",
}

In [ ]:
# ("id:ID",":LABEL","fullName","simpleName","color","dependencyProfileCategory","cohesion")
modules = [(id, 'Module', id, node['properties']['simpleName'], roleStereotypeColors[node['properties'].get('roleStereotype', 'Unknown')], dependencyProfiles.get(id, None), None)
                    for id,node in nodes.items() if 'Structure' in node['labels'] and id != 'java.lang.String']

modules

In [ ]:
import csv

with open(f"{project_name}-nodes.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(node_header)
    writer.writerow(domain_node)
    writer.writerows(component_nodes)
    writer.writerows(sublayers)
    writer.writerows(modules)

with open(f"{project_name}-edges.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(edge_header)
    writer.writerows(contains)
    writer.writerows(calls)